In [1]:
!pip install gensim

In [2]:
import os
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gensim 
import os
from os import listdir
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
import logging
import time
import re
import io
from nltk.corpus import stopwords
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
data = pd.read_csv('movies_metadata.csv',sep=',')

/home/baban/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
#We will consider movie name and overview only to predict
title = list(data['original_title'])
overview = data['overview'].tolist()

In [9]:

lemma = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())

# Remove numeric values, stopwords. Then lemmatize and stem the words
def filt(docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(docs))]
    lemma_docs = [ '' for i in range(len(docs))]
    for i in range(len(docs)):
        if i%100 == 0:
            print("Proceessing : "+ str(i))
        x= str(docs[i])
        words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", x)
        filtered_word_list = [word.lower() for word in words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        for word in filtered_word_list:
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs
overview_filtered = filt(overview)

[nltk_data] Downloading package stopwords to /home/baban/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/baban/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/baban/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Proceessing : 0
Proceessing : 100
Proceessing : 200
Proceessing : 300
Proceessing : 400
Proceessing : 500
Proceessing : 600
Proceessing : 700
Proceessing : 800
Proceessing : 900
Proceessing : 1000
Proceessing : 1100
Proceessing : 1200
Proceessing : 1300
Proceessing : 1400
Proceessing : 1500
Proceessing : 1600
Proceessing : 1700
Proceessing : 1800
Proceessing : 1900
Proceessing : 2000
Proceessing : 2100
Proceessing : 2200
Proceessing : 2300
Proceessing : 2400
Proceessing : 2500
Proceessing : 2600
Proceessing : 2700
Proceessing : 2800
Proceessing : 2900
Proceessing : 3000
Proceessing : 3100
Proceessing : 3200
Proceessing : 3300
Proceessing : 3400
Proceessing : 3500
Proceessing : 3600
Proceessing : 3700
Proceessing : 3800
Proceessing : 3900
Proceessing : 4000
Proceessing : 4100
Proceessing : 4200
Proceessing : 4300
Proceessing : 4400
Proceessing : 4500
Proceessing : 4600
Proceessing : 4700
Proceessing : 4800
Proceessing : 4900
Proceessing : 5000
Proceessing : 5100
Proceessing : 5200
Proce

Proceessing : 42700
Proceessing : 42800
Proceessing : 42900
Proceessing : 43000
Proceessing : 43100
Proceessing : 43200
Proceessing : 43300
Proceessing : 43400
Proceessing : 43500
Proceessing : 43600
Proceessing : 43700
Proceessing : 43800
Proceessing : 43900
Proceessing : 44000
Proceessing : 44100
Proceessing : 44200
Proceessing : 44300
Proceessing : 44400
Proceessing : 44500
Proceessing : 44600
Proceessing : 44700
Proceessing : 44800
Proceessing : 44900
Proceessing : 45000
Proceessing : 45100
Proceessing : 45200
Proceessing : 45300
Proceessing : 45400


In [10]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=re.split('\W+',doc), tags=[self.labels_list[idx]])
iterator = DocIterator(overview_filtered, title)

In [11]:
model = gensim.models.Doc2Vec(vector_size=150, window=10, min_count=2, workers=4, alpha=0.025, min_alpha=0.001)
model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')
tic = time.time()
model.train(iterator,total_examples=model.corpus_count, epochs = 50)
toc = time.time()
print(toc-tic)
model.save("doc2vec_Movie_recommender_vec_150_window_10_mc_2_epochs_50.model")

2019-08-30 18:05:45,342 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-08-30 18:05:45,346 : INFO : collecting all words and their counts
2019-08-30 18:05:45,348 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-08-30 18:05:45,614 : INFO : PROGRESS: at example #10000, processed 313484 words (1193572/s), 28322 word types, 9768 tags
2019-08-30 18:05:45,839 : INFO : PROGRESS: at example #20000, processed 624237 words (1379116/s), 41629 word types, 19334 tags
2019-08-30 18:05:46,068 : INFO : PROGRESS: at example #30000, processed 935119 words (1366416/s), 51266 word types, 28696 tags
2019-08-30 18:05:46,300 : INFO : PROGRESS: at example #40000, processed 1256548 words (1386133/s), 61832 word types, 38182 tags
2019-08-30 18:05:46,457 : INFO : collected 66181 word types and 43373 unique tags from a corpus of 45466 examples and 1418579 words
2019-08-30 18:05:46,458 : INFO : Loading a fresh vocabulary
2019-08-30 18:05:4

done building vocabulary
start training the model


2019-08-30 18:05:48,867 : INFO : EPOCH 1 - PROGRESS: at 23.13% examples, 300382 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:05:49,875 : INFO : EPOCH 1 - PROGRESS: at 44.95% examples, 298467 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:05:50,947 : INFO : EPOCH 1 - PROGRESS: at 68.24% examples, 297897 words/s, in_qsize 8, out_qsize 0
2019-08-30 18:05:52,010 : INFO : EPOCH 1 - PROGRESS: at 93.03% examples, 304781 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:05:52,160 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-08-30 18:05:52,165 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-30 18:05:52,254 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-30 18:05:52,279 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-30 18:05:52,282 : INFO : EPOCH - 1 : training on 1418579 raw words (1370415 effective words) took 4.5s, 306042 effective words/s
2019-08-30 18:05:53,330 : INFO : EPOCH 2 - PROG

2019-08-30 18:06:34,248 : INFO : EPOCH - 8 : training on 1418579 raw words (1370226 effective words) took 5.9s, 230806 effective words/s
2019-08-30 18:06:35,278 : INFO : EPOCH 9 - PROGRESS: at 18.28% examples, 247889 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:06:36,304 : INFO : EPOCH 9 - PROGRESS: at 34.59% examples, 232095 words/s, in_qsize 8, out_qsize 0
2019-08-30 18:06:37,365 : INFO : EPOCH 9 - PROGRESS: at 40.14% examples, 177573 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:06:38,409 : INFO : EPOCH 9 - PROGRESS: at 56.94% examples, 188670 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:06:39,411 : INFO : EPOCH 9 - PROGRESS: at 73.14% examples, 194986 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:06:40,547 : INFO : EPOCH 9 - PROGRESS: at 93.03% examples, 203982 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:06:40,679 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-08-30 18:06:40,683 : INFO : worker thread finished; awaiting finish of 2 more threads
201

2019-08-30 18:07:21,659 : INFO : EPOCH 16 - PROGRESS: at 48.46% examples, 318536 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:07:22,666 : INFO : EPOCH 16 - PROGRESS: at 72.41% examples, 320753 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:07:23,675 : INFO : EPOCH 16 - PROGRESS: at 98.29% examples, 328494 words/s, in_qsize 3, out_qsize 1
2019-08-30 18:07:23,677 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-08-30 18:07:23,681 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-30 18:07:23,713 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-30 18:07:23,717 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-30 18:07:23,717 : INFO : EPOCH - 16 : training on 1418579 raw words (1370361 effective words) took 4.2s, 330178 effective words/s
2019-08-30 18:07:24,736 : INFO : EPOCH 17 - PROGRESS: at 20.37% examples, 278280 words/s, in_qsize 8, out_qsize 0
2019-08-30 18:07:25,790 : INFO : EPOCH 17 

...

2019-08-30 18:08:40,519 : INFO : EPOCH 33 - PROGRESS: at 68.24% examples, 297889 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:08:41,529 : INFO : EPOCH 33 - PROGRESS: at 93.04% examples, 308648 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:08:41,700 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-08-30 18:08:41,710 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-30 18:08:41,783 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-30 18:08:41,790 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-30 18:08:41,791 : INFO : EPOCH - 33 : training on 1418579 raw words (1370615 effective words) took 4.4s, 310323 effective words/s
2019-08-30 18:08:42,827 : INFO : EPOCH 34 - PROGRESS: at 23.13% examples, 312348 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:08:43,885 : INFO : EPOCH 34 - PROGRESS: at 48.46% examples, 320407 words/s, in_qsize 8, out_qsize 0
2019-08-30 18:08:44,926 : INFO : EPOCH 34 

2019-08-30 18:09:17,988 : INFO : EPOCH - 41 : training on 1418579 raw words (1370552 effective words) took 4.6s, 299549 effective words/s
2019-08-30 18:09:19,016 : INFO : EPOCH 42 - PROGRESS: at 14.05% examples, 191314 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:09:20,019 : INFO : EPOCH 42 - PROGRESS: at 32.48% examples, 220749 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:09:21,089 : INFO : EPOCH 42 - PROGRESS: at 56.94% examples, 253560 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:09:22,094 : INFO : EPOCH 42 - PROGRESS: at 79.85% examples, 268899 words/s, in_qsize 7, out_qsize 0
2019-08-30 18:09:22,774 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-08-30 18:09:22,779 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-30 18:09:22,814 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-30 18:09:22,819 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-30 18:09:22,820 : INFO : EPOCH - 4

2019-08-30 18:09:58,094 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-08-30 18:09:58,125 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-08-30 18:09:58,130 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-30 18:09:58,130 : INFO : EPOCH - 50 : training on 1418579 raw words (1370491 effective words) took 4.2s, 327472 effective words/s
2019-08-30 18:09:58,131 : INFO : training on a 70928950 raw words (68523089 effective words) took 250.3s, 273725 effective words/s
2019-08-30 18:09:58,131 : INFO : saving Doc2Vec object under doc2vec_Movie_recommender_vec_150_window_10_mc_2_epochs_50.model, separately None


250.33632636070251


2019-08-30 18:09:59,055 : INFO : saved doc2vec_Movie_recommender_vec_150_window_10_mc_2_epochs_50.model


In [27]:
model.docvecs.most_similar(title[0])

TypeError: ignored